<table>
  <tr>
    <td>Guy Uliel</td>
    <td>Rotem Dahan</td>
  </tr>
  <tr>
    <td>318439908</td>
    <td>316222215</td>
  </tr>
</table>


[GitHub Link](https://github.com/guyuliel/Data-Science-Project)


In [1]:
import requests
from bs4 import BeautifulSoup
from datetime import datetime
import calendar

In [2]:
import pandas as pd 

In [3]:
import urllib.request
import urllib.parse
import json

### Scraping Data


##### We will insert the data into a list of dictionaries - each dictionary in the list represent an ad

In [4]:
def get_html(url):  # function that gets an HTML and response it back (Unreadable)
    try:
        response = requests.get(url)
        response.raise_for_status()  # Raise an error for bad status codes
        return response.text
    except requests.RequestException as e:
        print(f"Error fetching URL {url}: {e}")
        return None

def get_ad_urls(url):  # function that gets a URL (page) and for each page find all the URL ads and save them in a list
    try:
        html_content = get_html(url)
        if html_content is None:
            return []
        soup = BeautifulSoup(html_content, 'html.parser')
        cards = soup.find_all('div', class_='card-block')

        ad_urls = []
        for card in cards:
            # Stop processing once we reach the footer section
            if card.find_parent('footer'):
                break
            ad_url_tag = card.find('a', href=True)
            if ad_url_tag:
                ad_urls.append('https://www.ad.co.il' + ad_url_tag['href'])

        return ad_urls
    except Exception as e:
        print(f"Error parsing ad URLs from {url}: {e}")
        return []

def data_scrape(url):  # function that gets a URL and scrapes all the relevant data into a dictionary
    try:
        html_content = get_html(url)
        if html_content is None:
            return {}
        soup = BeautifulSoup(html_content, 'html.parser')

        vehicle_data = {}

        # Extract details from the table
        try:
            table = soup.find('table', class_='table table-sm mb-4')
            if table:
                rows = table.find_all('tr')
                for row in rows:
                    key_element = row.find_all('td')[0].text.strip()
                    value_element = row.find_all('td')[1].text.strip()
                    if key_element in ['ק"מ', 'נפח']:
                        value_element = value_element.replace(',', '')
                    vehicle_data[key_element] = value_element
        except AttributeError:
            print(f"Table not found on page {url}")

        # Extract manufacturer, model, and price
        try:
            card_body = soup.find('div', class_='card-body p-3')
            if card_body:
                titles = card_body.find_all('h2', class_='card-title')
                if len(titles) >= 2:
                    title_text = titles[0].text.strip()
                    price_text = titles[1].text.strip()
                    # Assuming the title contains the manufacturer and model
                    title_parts = title_text.split()
                    if len(title_parts) >= 2:
                        vehicle_data['יצרן'] = title_parts[0]  # Assuming the first word is the manufacturer
                        vehicle_data['דגם'] = ' '.join(title_parts[1:])  # The rest is the model
                    vehicle_data['מחיר'] = price_text.replace('₪', '').replace(',', '').strip()  # Remove the currency symbol, commas, and whitespace
        except AttributeError:
            print(f"Card body not found on page {url}")

        # Count the number of images in the gallery
        try:
            gallery_div = soup.find('div', class_='col-12 d-flex mt-3 justify-content-center flex-wrap', itemscope=True)
            if gallery_div:
                images = gallery_div.find_all('img')
                vehicle_data['מספר תמונות'] = len(images)
        except AttributeError:
            print(f"Gallery not found on page {url}")

        # Extract creation date and last bump date
        try:
            date_divs = soup.find_all('div', class_='px-3')
            if date_divs:
                for date_div in date_divs:
                    text = date_div.text.strip()
                    if 'תאריך יצירה' in text:
                        vehicle_data['תאריך יצירה'] = text.split(':')[-1].strip()
                    elif 'תאריך הקפצה אחרון' in text:
                        vehicle_data['תאריך הקפצה אחרון'] = text.split(':')[-1].strip()
        except AttributeError:
            print(f"Date divs not found on page {url}")

        # Extract description and clean it up
        try:
            meta_description = soup.find('meta', attrs={'name': 'description'})
            if meta_description and 'content' in meta_description.attrs:
                description = meta_description['content'].strip()
                # Clean up the description by replacing newline characters with spaces and reducing multiple spaces to a single space
                clean_description = ' '.join(description.split())
                vehicle_data['תיאור'] = clean_description
        except AttributeError:
            print(f"Meta description not found on page {url}")

        # Convert 'Test' date to days until the last day of the month
        try:
            test_date_str = vehicle_data.get('טסט עד')
            if test_date_str:
                test_date = datetime.strptime(test_date_str, '%m/%Y')
                last_day_of_month = datetime(test_date.year, test_date.month, calendar.monthrange(test_date.year, test_date.month)[1])
                days_until_last_day = (last_day_of_month - datetime.now()).days
                vehicle_data['טסט עד'] = days_until_last_day
        except (ValueError, TypeError):
            vehicle_data['טסט עד'] = 'Invalid date'

        return vehicle_data
    except Exception as e:
        print(f"Error scraping data from {url}: {e}")
        return {}

def get_next_page_url(soup):  # function that gets the next page URL
    try:
        next_page_tag = soup.find('li', class_='nextPage')
        if next_page_tag:
            next_page_link = next_page_tag.find('a', href=True)
            if next_page_link and 'href' in next_page_link.attrs:
                return 'https://www.ad.co.il' + next_page_link['href']
        return None
    except Exception as e:
        print(f"Error finding next page URL: {e}")
        return None


In [5]:
base_url = 'https://www.ad.co.il/car?sp261=13905'  # URL for Mitsubishi cars
url = base_url
mitsu = [] # the list of the dictionaries - each dictionary is an ad

while url != None:
    html_content = get_html(url) # we get the html file (unreadble) by using the function 'get_html'
    soup = BeautifulSoup(html_content, 'html.parser') # make it readable( HTML document) 
    ad_urls = get_ad_urls(url) # we get into list all the urls of the page i

    for ad_url in ad_urls:
        vehicle_data = data_scrape(ad_url) # we scrape each ad for page i (as a dictionary)
        mitsu.append(vehicle_data) # we add it into the list 
    # Get the URL of the next page
    url = get_next_page_url(soup) # we change the value of the url to be index + 1 (next page) 
    # And we go back to the while loop to do the same as we did - with the next page



In [ ]:
mitsu

In [7]:
# Convert the list of dictionaries to a DataFrame
scrape_df = pd.DataFrame(mitsu)


# Define the desired column order using key_mapping values
desired_columns = {
    'יצרן': 'manufactor',
    'שנה': 'Year',
    'דגם': 'model',
    'יד': 'Hand',
    'ת. הילוכים': 'Gear',
    'נפח': 'Engine_capacity',
    'סוג מנוע': 'Engine_type',
    'בעלות קודמת': 'Prev_ownership',
    'בעלות נוכחית': 'Curr_ownership',
    'אזור': 'Area',
    'עיר': 'City',
    'מחיר': 'Price',
    'מספר תמונות': 'Pic_num',
    'תאריך יצירה': 'Cre_date',
    'תאריך הקפצה אחרון': 'Repub_date',
    'תיאור': 'Description',
    'צבע': 'Color',
    'ק"מ': 'Km',
    'טסט עד': 'Test'
}
# Add 'Supply_score' column 
scrape_df['Supply_score'] = pd.NA

# Reindex the DataFrame to match the desired column order
scrape_df = scrape_df.reindex(columns=list(desired_columns.keys())+['Supply_score'])

# rename the columns to English
scrape_df.rename(columns=desired_columns, inplace=True)

#### Converting the types of the colunms 

In [9]:
int_lst = ['Year','Hand','Engine_capacity','Pic_num','Km','Test','Supply_score']
str_lst = ['manufactor','model','Area','City','Description','Color']
date_lst = ['Cre_date','Repub_date']
cat_lst = ['Gear','Engine_type','Prev_ownership','Curr_ownership']

In [11]:
scrape_df[int_lst] = scrape_df[int_lst].apply(pd.to_numeric, errors='coerce').astype('Int64')
scrape_df["Price"] = scrape_df["Price"].apply(pd.to_numeric, errors='coerce').astype(float)
scrape_df[date_lst] = scrape_df[date_lst].apply(lambda x: pd.to_datetime(x, format='%d/%m/%Y', errors='coerce'))
scrape_df[str_lst] = scrape_df[str_lst].astype('str')
scrape_df[cat_lst] = scrape_df[cat_lst].astype('category')

In [12]:
scrape_df

,manufactor,Year,model,Hand,Gear,Engine_capacity,Engine_type,Prev_ownership,Curr_ownership,Area,City,Price,Pic_num,Cre_date,Repub_date,Description,Color,Km,Test,Supply_score
0,מיצובישי,2005,לנסר קלאסיק,8,אוטומטית,1600,בנזין,פרטית,פרטית,ירושלים והסביבה,ירושלים,4700.0,3,2024-05-22,2024-05-27,רכב במצב פצצה שירת אותי נאמנה הוחלף לפני חודש ...,כסוף,230000,140,<NA>
1,מיצובישי,2016,GT3000,2,אוטומטית,2000,בנזין,פרטית,פרטית,חיפה וחוף הכרמל,נשר,92000.0,4,2024-05-05,2024-05-05,אאוטלנדר המפואר פרמיום 7 מקומות טופל אצל היבוא...,לבן,141000,<NA>,<NA>
2,מיצובישי,2016,I-MIEV,2,אוטומטית,2000,בנזין,פרטית,פרטית,חיפה וחוף הכרמל,נשר,92000.0,4,2024-05-05,2024-05-04,אאוטלנדר הדגם המפואר 7 מקומות שמור ומטופל אצל ...,לבן,141000,260,<NA>
3,מיצובישי,2020,אקליפס,2,אוטומטית,2000,בנזין,ליסינג,פרטית,חיפה וחוף הכרמל,טירת כרמל,85000.0,3,2024-04-25,2024-04-25,"מיצובישי ASX 76,000 ק""מ מוכן גם להחליף.",שחור,76000,109,<NA>
4,מיצובישי,2010,לנסר,3,אוטומטית,1600,בנזין,פרטית,פרטית,נתניה והסביבה,נתניה,10000.0,7,2024-03-16,2024-03-16,גיר מנוע תקינים מעככת מוביליין רכב 7 שנים עצלי...,כסוף,242000,170,<NA>
...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...
154,מיצובישי,2017,אאוטלנדר,1,אוטומטית,1998,בנזין,NaN,NaN,מודיעין והסביבה,מודיעין מכבים רעות,106000.0,<NA>,2022-05-22,2022-05-22,"מצב מצויין, שמור, כל הטיפולים בסוכנות מיצובישי...",nan,122000,<NA>,<NA>
155,מיצובישי,2020,אקליפס,1,אוטומטית,1500,בנזין,NaN,NaN,מודיעין והסביבה,מודיעין,129000.0,<NA>,2022-05-11,2022-05-11,רכב שמור נמצא בחניה מקורה נהג יחיד(70),nan,16000,<NA>,<NA>
156,מיצובישי,2010,לנסר הדור החדש,3,אוטומטית,1500,בנזין,NaN,NaN,קריות,רכסים,11500.0,<NA>,2022-05-05,2022-05-05,חלונות שחורים מסך באגז גדול מושבים יפים,nan,220000,<NA>,<NA>
157,מיצובישי,2017,אאוטלנדר,2,אוטומטית,2000,בנזין,NaN,NaN,באר שבע והסביבה,באר שבע,115000.0,<NA>,2022-04-26,2022-04-26,הרכב במצב מצויין טסט שלמה שלמה צלחות ברקסים וצ...,nan,85000,<NA>,<NA>


In [13]:
scrape_df.dtypes

manufactor                 object
Year                        Int64
model                      object
Hand                        Int64
Gear                     category
Engine_capacity             Int64
Engine_type              category
Prev_ownership           category
Curr_ownership           category
Area                       object
City                       object
Price                     float64
Pic_num                     Int64
Cre_date           datetime64[ns]
Repub_date         datetime64[ns]
Description                object
Color                      object
Km                          Int64
Test                        Int64
Supply_score                Int64
dtype: object

### Getting Data from API


In [14]:
def fetch_filtered_data_from_api(api_url, resource_id, query, keys_to_extract, limit=1000):
    offset = 0
    all_records = []

    while True:
        # Construct the URL with limit, offset, and query parameters
        params = {
            'resource_id': resource_id,
            'limit': limit,
            'offset': offset,
            'q': query
        }
        url = f"{api_url}?{urllib.parse.urlencode(params)}"
        

        # Make a GET request to the API
        with urllib.request.urlopen(url) as response:
            if response.status == 200:
                data = json.loads(response.read().decode())
                
                # Check if the response contains the 'result' key
                if 'result' in data:
                    records = data['result']['records']
                    
                    if not records:
                        break
                    
                    # Filter the records to include only the specified keys
                    filtered_records = [
                        {key: record.get(key, 'NA') for key in keys_to_extract}
                        for record in records
                    ]
                    
                    # Add the filtered records to the all_records list
                    all_records.extend(filtered_records)
                    
                    # Update the offset for the next iteration
                    offset += limit
                else:
                    print("No 'result' key found in the API response.")
                    break
            else:
                print(f"Failed to fetch data from API. Status code: {response.status}")
                break

    return all_records
# Base URL for the API
api_url = 'https://data.gov.il/api/3/action/datastore_search'
    
# Resource ID for the dataset
resource_id = '5e87a7a1-2f6f-41c1-8aec-7216d52a6cf6'
    
# Query parameter for Mitsubishi vehicles
query = 'מיצובישי'
    
# Keys to extract from the API response
keys_to_extract = ['tozar', 'kinuy_mishari', 'shnat_yitzur']
    
# Fetch filtered data from the API
mitsu1 = fetch_filtered_data_from_api(api_url, resource_id, query, keys_to_extract)

In [15]:
mitsu1

[{'tozar': 'מיצובישי',
  'kinuy_mishari': 'מיצובישי סופר לנסר I',
  'shnat_yitzur': 1997},
 {'tozar': 'מיצובישי',
  'kinuy_mishari': 'מיצובישי סופר לנסר I',
  'shnat_yitzur': 1996},
 {'tozar': 'מיצובישי',
  'kinuy_mishari': 'מיצובישי סופר לנסר I',
  'shnat_yitzur': 1996},
 {'tozar': 'מיצובישי',
  'kinuy_mishari': 'מיצובישי סופר לנסר I',
  'shnat_yitzur': 1997},
 {'tozar': 'מיצובישי', 'kinuy_mishari': 'OUTLANDER', 'shnat_yitzur': 2008},
 {'tozar': 'מיצובישי', 'kinuy_mishari': 'SPACE STAR', 'shnat_yitzur': 2020},
 {'tozar': 'מיצובישי', 'kinuy_mishari': 'L200', 'shnat_yitzur': 2014},
 {'tozar': 'מיצובישי', 'kinuy_mishari': 'L 200', 'shnat_yitzur': 2007},
 {'tozar': 'מיצובישי', 'kinuy_mishari': 'GALANT', 'shnat_yitzur': 2002},
 {'tozar': 'מיצובישי', 'kinuy_mishari': 'GALANT', 'shnat_yitzur': 2001},
 {'tozar': 'מיצובישי', 'kinuy_mishari': 'ASX', 'shnat_yitzur': 2019},
 {'tozar': 'מיצובישי', 'kinuy_mishari': 'SPACE WAGON', 'shnat_yitzur': 2002},
 {'tozar': 'מיצובישי', 'kinuy_mishari': 'ECLIP

##### We are going to mapping the model values from mitsu1 to be suited to our main data (scrape_df)

In [16]:
kinuy_mishari_mappings = {
    'OUILANDER': 'אאוטלנדר',
    'OUTLANDER': 'אאוטלנדר',
    'LANCER': 'לנסר',
    'ECLIPSE CROSS': 'אקליפס',
    'SPACE STAR': 'ספייס סטאר',
    'LANCER EVOLUTIO': 'לנסר איוולושן',
    'LANCER SPORTBAC': 'לנסר ספורטבק',
    'GRANDIS': 'גרנדיס',
    'ATTRAGE': "אטראז'",
    'COLT': 'קולט',
    'מיצובישי סופר לנסר I': 'סופר לנסר',
    'סופר לנסר ILG סלון': 'סופר לנסר'
}

In [17]:
for record in mitsu1:
    kinuy_mishari = record.get('kinuy_mishari', '')
    if kinuy_mishari in kinuy_mishari_mappings:
        record['kinuy_mishari'] = kinuy_mishari_mappings[kinuy_mishari]


In [18]:
api_df = pd.DataFrame(mitsu1)

In [19]:
api_df

,tozar,kinuy_mishari,shnat_yitzur
0,מיצובישי,סופר לנסר,1997
1,מיצובישי,סופר לנסר,1996
2,מיצובישי,סופר לנסר,1996
3,מיצובישי,סופר לנסר,1997
4,מיצובישי,אאוטלנדר,2008
...,...,...,...
1602,מיצובישי,ASX,2021
1603,מיצובישי,ספייס סטאר,2013
1604,מיצובישי,L200,2014
1605,מיצובישי,אטראז',2017


In [20]:
agg_api_df = api_df.groupby(['tozar', 'kinuy_mishari', 'shnat_yitzur']).size().reset_index(name='Supply_score')

In [21]:
agg_api_df

,tozar,kinuy_mishari,shnat_yitzur,Supply_score
0,מיצובישי,004-L מיניבוס בנזין,1998,1
1,מיצובישי,004-L קומבי דיזל ידנ,1998,1
2,מיצובישי,004-L קומבי דיזל ידנ,1999,1
3,מיצובישי,ASX,2011,2
4,מיצובישי,ASX,2017,3
...,...,...,...,...
260,מיצובישי,קומבי דיזל 1+3,1998,1
261,מיצובישי,קומבי דיזל 2+3,1998,1
262,מיצובישי,קומבי דיזל 2+6,1998,1
263,מיצובישי,קומבי דיזל 2+6 אוטוב,1998,1


In [22]:
api_df.dtypes

tozar            object
kinuy_mishari    object
shnat_yitzur      int64
dtype: object

### Merging the 2 main DF - scrape_df & agg_api_df

In [23]:
agg_api_df.rename(columns={'kinuy_mishari': 'model', 'shnat_yitzur': 'Year'}, inplace=True)

# Merge DataFrames on the specified columns
merged_df = pd.merge(scrape_df, agg_api_df[['model', 'Year', 'Supply_score']], on=['model', 'Year'], how='left')

# Ensure Supply_score is Int64 
merged_df['Supply_score'] = merged_df['Supply_score_y'].fillna(merged_df['Supply_score_x']).astype('Int64')
merged_df.drop(columns=['Supply_score_x', 'Supply_score_y'], inplace=True)

In [24]:
merged_df.dtypes

manufactor                 object
Year                        Int64
model                      object
Hand                        Int64
Gear                     category
Engine_capacity             Int64
Engine_type              category
Prev_ownership           category
Curr_ownership           category
Area                       object
City                       object
Price                     float64
Pic_num                     Int64
Cre_date           datetime64[ns]
Repub_date         datetime64[ns]
Description                object
Color                      object
Km                          Int64
Test                        Int64
Supply_score                Int64
dtype: object

### Slicing the Data between 2017-2024


In [25]:
mydf = merged_df[(merged_df['Year'] >= 2017) & (merged_df['Year'] <= 2024)]

In [26]:
mydf

,manufactor,Year,model,Hand,Gear,Engine_capacity,Engine_type,Prev_ownership,Curr_ownership,Area,City,Price,Pic_num,Cre_date,Repub_date,Description,Color,Km,Test,Supply_score
3,מיצובישי,2020,אקליפס,2,אוטומטית,2000,בנזין,ליסינג,פרטית,חיפה וחוף הכרמל,טירת כרמל,85000.0,3,2024-04-25,2024-04-25,"מיצובישי ASX 76,000 ק""מ מוכן גם להחליף.",שחור,76000,109,15
5,מיצובישי,2022,אקליפס,1,אוטומטית,1500,בנזין,פרטית,פרטית,נס ציונה - רחובות,רחובות,129000.0,6,2024-02-06,2024-02-06,דגם executive מנוע 1.5 טורבו מפתח חכם כניסה וה...,כחול מטאלי,21500,<NA>,15
6,מיצובישי,2020,לנסר ספורטבק,2,אוטומטית,4200,גז,פרטית,פרטית,באר שבע והסביבה,באר שבע,35000.0,4,2024-01-05,2024-01-05,2020 מיצובישי אאוטלנדר ספורט ES 4x4 1-בעלים מנ...,לבן,90000,<NA>,<NA>
7,מיצובישי,2020,לנסר ספורטבק,2,אוטומטית,4200,גז,פרטית,פרטית,חדרה וישובי עמק חפר,חדרה,35000.0,4,2024-01-05,2024-01-05,2020 מיצובישי אאוטלנדר ספורט ES 4x4 1-בעלים מנ...,לבן,90000,<NA>,<NA>
8,מיצובישי,2020,לנסר ספורטבק,2,אוטומטית,4200,גז,פרטית,פרטית,באר שבע והסביבה,דימונה,35000.0,4,2024-01-05,2024-01-05,2020 מיצובישי אאוטלנדר ספורט ES 4x4 1-בעלים מנ...,לבן,90000,<NA>,<NA>
...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...
151,מיצובישי,2018,ASX,1,אוטומטית,2000,בנזין,NaN,NaN,"ראשל""צ והסביבה",בית דגן,90000.0,<NA>,2022-07-15,2022-07-15,רכב שמור מטופל במוסכי מיטצובישי בלבד. לא עבור ...,nan,102000,<NA>,5
152,מיצובישי,2019,ASX,1,אוטומטית,2000,בנזין,NaN,NaN,nan,יקנעם עילית,107000.0,<NA>,2022-07-14,2022-07-14,"שמורה מאד,מטופלת אך ורק במוסך מורשה ללא תאונות",nan,69000,<NA>,6
154,מיצובישי,2017,אאוטלנדר,1,אוטומטית,1998,בנזין,NaN,NaN,מודיעין והסביבה,מודיעין מכבים רעות,106000.0,<NA>,2022-05-22,2022-05-22,"מצב מצויין, שמור, כל הטיפולים בסוכנות מיצובישי...",nan,122000,<NA>,26
155,מיצובישי,2020,אקליפס,1,אוטומטית,1500,בנזין,NaN,NaN,מודיעין והסביבה,מודיעין,129000.0,<NA>,2022-05-11,2022-05-11,רכב שמור נמצא בחניה מקורה נהג יחיד(70),nan,16000,<NA>,15


### Export to CSV File

In [27]:
mydf.to_csv('mitsu17_24.csv',encoding='utf-8-sig',index=False)